# Conditional Deep Convolutional Generative Adversarial Network (cDCGAN)
## Tutorial

In [ ]:
import numpy as np
import pickle
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import datetime
from tensorflow.keras import layers, models
import tensorflow.keras.backend as kb


from IPython import display
import warnings; warnings.simplefilter('ignore')

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
#get some utils from:
!git clone https://github.com/dlanci/UZHMLWorkshop2020-GAN
os.chdir('UZHMLWorkshop2020-GAN/')
from utils.utils import load_dataset
from utils.utils import generate_and_save_images_conditional

In [ ]:
#load the dataset file
!wget -q --show-progress -O batch0.pickle "https://www.dropbox.com/s/wn8ilvp8k67grz4/batch0.pickle?dl=0";
!wget -q --show-progress -O batch1.pickle "https://www.dropbox.com/s/5vjme0o8drbi0v4/batch1.pickle?dl=0";
!wget -q --show-progress -O batch2.pickle "https://www.dropbox.com/s/rz2b8c4911kb4iy/batch2.pickle?dl=0";
!wget -q --show-progress -O batch3.pickle "https://www.dropbox.com/s/2wa94zzt7wq2002/batch3.pickle?dl=0";
!wget -q --show-progress -O batch4.pickle "https://www.dropbox.com/s/icntmsval8nync5/batch4.pickle?dl=0";
tuple_, tot_evts = load_dataset(path='.', test=False)

## 2. The dataset

### 2.1 Load the images dataset

In [ ]:


X_pixels=48
Y_pixels=48


X_train=tuple_['reco_imgs'][0:np.int(tot_evts*0.9)]
X_test=tuple_['reco_imgs'][np.int(tot_evts*0.9):tot_evts]


maxval=X_train[np.where(X_train!=0)].max()

X_train_norm = (X_train)/(maxval)
X_test_norm = (X_test)/(maxval)


plt.subplot(1,2,1)
plt.imshow(X_train_norm[3].reshape(X_pixels,Y_pixels))
plt.xlabel('X', fontsize=15)
plt.ylabel('Y', fontsize=15)
plt.title('A sample image', fontsize=15)
plt.colorbar()
plt.subplot(1,2,2)
plt.hist(np.sum(X_train_norm,axis=(1,2,3)),bins=100);
plt.xlabel('E (a.u.)', fontsize=15)
plt.ylabel('dN/dE', fontsize=15)
plt.title('Total energy deposit per image', fontsize=15)
fig = plt.gcf()
fig.set_size_inches(16,5)



### 2.2 Load the labels dataset

In [ ]:
y_train=tuple_['all_tracks'][0:np.int(tot_evts*0.9)]
y_test=tuple_['all_tracks'][np.int(tot_evts*0.9):tot_evts]



y_train1=np.where(y_train[:,0:24]<0,0.,y_train[:,0:24])
y_train2=np.where(y_train1[:,0:24]>48,0.,y_train1[:,0:24])
y_train_norm=np.zeros_like(y_train)

y_train_norm[:,0:24]=(y_train2)/48


ymaxE = 6120
y_train_norm[:,24:36]=(y_train[:,24:36])/(ymaxE)

y_test1=np.where(y_test[:,0:24]<0,0,y_test[:,0:24])
y_test2=np.where(y_test1[:,0:24]>48,0,y_test1[:,0:24])
y_test_norm=np.zeros_like(y_test)
y_test_norm[:,0:24]=(y_test2)/48

y_test_norm[:,24:36]=(y_test[:,24:36])/(ymaxE)


plt.subplot(1,2,1)
plt.hist(y_train_norm[:,0:12].flatten(),range=(0.001,1.1),bins=100);
plt.xlabel('dN/dpX', fontsize=15)
plt.ylabel('pX', fontsize=15)
plt.title('pX distr', fontsize=15)
plt.subplot(1,2,2)
plt.hist(y_train_norm[:,24:36].flatten(),range=(0.001,1.1),bins=100);
plt.xlabel('real E (a.u.)', fontsize=15)
plt.ylabel('dN/dE', fontsize=15)
plt.title('real E distr', fontsize=15)
fig = plt.gcf()
fig.set_size_inches(16,5)




### 2.3 Create Tensorflow Datasets


In [ ]:
BATCH_SIZE = 32
BUFFER_TRAIN_SIZE = np.int(tot_evts*0.9)
BUFFER_TEST_SIZE = np.int(tot_evts*0.1)



X_train_dataset = tf.data.Dataset.from_tensor_slices(X_train_norm)
label_train_dataset = tf.data.Dataset.from_tensor_slices(y_train_norm)

train_dataset=tf.data.Dataset.zip((X_train_dataset, label_train_dataset))
train_dataset=train_dataset.shuffle(BUFFER_TRAIN_SIZE, 
                                      reshuffle_each_iteration=True)


train_dataset=train_dataset.batch(BATCH_SIZE, drop_remainder=True)




it = iter(train_dataset)
img, label = next(it)
plt.imshow(img.numpy()[0].reshape(48,48))
print(label[0].numpy())

In [ ]:
X_test_dataset = tf.data.Dataset.from_tensor_slices(X_test_norm)
label_test_dataset = tf.data.Dataset.from_tensor_slices(y_test_norm)

test_dataset=tf.data.Dataset.zip((X_test_dataset, label_test_dataset))
test_dataset=test_dataset.shuffle(BUFFER_TEST_SIZE, 
                                      reshuffle_each_iteration=True)


test_dataset=test_dataset.batch(BATCH_SIZE, drop_remainder=True)

it = iter(train_dataset)
img, label = next(it)
plt.imshow(img.numpy()[0].reshape(48,48))
print(label[0].numpy())


## 3. Model building

### 3.1 Building the Generator model


In [ ]:
def make_generator_model(z_dim, label_dim):
    
    n_nodes = 12*12
    useb=False
    
    #initializer = tf.keras.initializers.TruncatedNormal(mean=0., stddev=1.)
    initializer = tf.keras.initializers.GlorotNormal()
    
    label_input_init = layers.Input((label_dim,))
    
    label_input = layers.Dense(64 * n_nodes, use_bias=useb,kernel_initializer=initializer)(label_input_init)
    label_input = layers.LeakyReLU(alpha=0.1)(label_input)
    label_input_as_img = layers.Reshape((12,12, 64))(label_input)

    
    z_input_init = layers.Input((z_dim,))
    z_input = layers.Dense(64 * n_nodes, use_bias=useb, kernel_initializer=initializer)(z_input_init)
    z_input = layers.LeakyReLU(alpha=0.1)(z_input)
    z_input = layers.Reshape((12, 12, 64))(z_input)


    hid = layers.Concatenate(axis=3)([label_input_as_img, z_input])

    hid = layers.Conv2DTranspose(filters=32,                           # number of output channels
                                 kernel_size=(4,4),                    # size of the convolution kernel              
                                 strides=(2,2),                        
                                 padding='same',
                                 use_bias=False)(hid)
    #print(hid.shape)

    
    hid=layers.BatchNormalization()(hid)
    hid=layers.LeakyReLU(alpha=0.2)(hid)
    
    
    hid = layers.Conv2DTranspose(filters=16,
                                 kernel_size=(5,5), 
                                 strides=(2,2), 
                                 padding='same',
                                 use_bias=False)(hid)
    
    #print(hid.shape)
    
    hid=layers.BatchNormalization()(hid)
    hid=layers.LeakyReLU(alpha=0.2)(hid)    
    
    out = layers.Conv2DTranspose(filters=1,
                                 kernel_size=(5,5), 
                                 strides=(1,1), 
                                 padding='same',
                                 use_bias=False,
                                 activation='sigmoid')(hid)

    #print(hid.shape)
    
    model = models.Model(inputs=[z_input_init,label_input_init], outputs=out)
    
    return model
    


In [ ]:
noise_dim=128 
generator = make_generator_model(z_dim=noise_dim, label_dim=36)
generator.summary()

### 3.2 Building the Discriminator model



In [ ]:
def make_discriminator_model(label_dim):
    
    #initializer = tf.keras.initializers.TruncatedNormal(mean=0., stddev=1.)
    initializer = tf.keras.initializers.GlorotNormal()    
    n_nodes = X_pixels*Y_pixels
    
    label_input_init = layers.Input((label_dim,))
    label_input = layers.Dense(n_nodes, kernel_initializer=initializer)(label_input_init)
    label_input = layers.LeakyReLU(alpha=0.2)(label_input)
    label_input = layers.Reshape((X_pixels,Y_pixels,1))(label_input)

    
    image_input_layer = layers.Input((X_pixels,Y_pixels,1))                     # Again define the input
    
    hid = layers.Concatenate(axis=3)([image_input_layer, label_input])

    hid = layers.Conv2D(64, kernel_size=4, strides=2, padding='same')(hid) # Example usage
    hid = layers.LeakyReLU(alpha=0.2)(hid)                                               # of a convolutional
    hid = layers.Dropout(0.3)(hid)                                                       # layer
    
    #print(hid.output)
    
    hid = layers.Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
    hid = layers.LeakyReLU(alpha=0.2)(hid)
    hid = layers.Dropout(0.3)(hid)
    
    #print(hid.output)
    
    feature = layers.Flatten()(hid)     #Here I call the output "feature" for future needs
    
    hid = layers.Dense(32)(feature)
    hid = layers.LeakyReLU(alpha=0.2)(hid)
    
    
    

    #dimensions of minibatch matrix
    n_kernels=12
    dim_kernel=12
    
    mb_discr = layers.Dense(n_kernels*dim_kernel)(hid)
    mb_discr = layers.Reshape((n_kernels,dim_kernel))(mb_discr)  #Matrix M
    
    diffs = kb.expand_dims(mb_discr, 3)-kb.expand_dims(kb.permute_dimensions(mb_discr, [1, 2, 0]), 0)
    abs_diffs = kb.sum(kb.abs(diffs), axis=2)
    
    minibatch_features = kb.sum(kb.exp(-abs_diffs),2)
    print(minibatch_features.shape)# (None, n_kernels)
    hid=layers.Concatenate()([hid, minibatch_features])

    
    out = layers.Dense(1)(hid) #note that we don't activate the last layer of the discriminator
                               #as the loss function we will use requires the unactivated output
    
    model = models.Model(inputs=[image_input_layer, label_input_init], outputs=[out, feature])
    
    
    return model
    

In [ ]:
discriminator = make_discriminator_model(36)
discriminator.summary()

In [ ]:
noise = tf.random.normal((1, noise_dim))
generated_image = generator([noise,label[0].numpy().reshape(-1,36)], training=False)
plt.imshow(generated_image[0,:,:,0])
plt.colorbar()

In [ ]:
decision, feature = discriminator([generated_image,label[0].numpy().reshape(-1,36)])
print (decision)

## 4 Definition of the Losses

In the next step we will define the functions to be minimized during the training process. The two models (G,D) participate in a non-cooperative game and try to minimize two different loss functions:


In [ ]:
def discriminator_loss(real_output, fake_output, eps=1e-3):

    real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                logits=real_output,
                #labels=tf.ones_like(real_output))) #one sided label smoothing
                labels=(1-eps)*tf.ones_like(real_output))) #one sided label smoothing
    
    
    fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                logits=fake_output,
                labels=tf.zeros_like(fake_output)))
    
    total_loss = real_loss + fake_loss
    
    return total_loss


discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)   #define the minimization algorithm here

In [ ]:
def generator_loss_GAN(fake_output, eps=1e-3):
    gan_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                logits=fake_output,
                #labels=tf.ones_like(fake_output))) 
                labels=(1-eps)*tf.ones_like(fake_output))) #one sided label smoothing
    return gan_loss
    
def generator_loss_CYCLIC(true_images, fake_images):
    
    cyclic_loss = tf.reduce_mean(tf.abs(tf.cast(fake_images,tf.float32)-tf.cast(true_images,tf.float32)))
      
    return cyclic_loss


def generator_loss_FEATURE(true_features, fake_features):
    gan_loss= tf.sqrt(tf.reduce_mean(tf.pow(true_features-fake_features,2)))
    return gan_loss

  

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4)   #define the minimization algorithm here

## 5 Set up the training cycle

### 5.1 Tensorboard for metrics inspection

In [ ]:
# Define our metrics

d_train_loss = tf.keras.metrics.Mean('d_train_loss', dtype=tf.float32)
g_train_loss = tf.keras.metrics.Mean('g_train_loss', dtype=tf.float32)
d_train_accuracy_on_real = tf.keras.metrics.BinaryCrossentropy('train_accuracy_on_real', from_logits=True)
d_train_accuracy_on_fake = tf.keras.metrics.BinaryCrossentropy('train_accuracy_on_fake', from_logits=True)

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
os.makedirs('./condGAN/logs/gradient_tape/', exist_ok=True)
train_log_dir = './condGAN/logs/gradient_tape/' + current_time + '/train'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)




### 5.2 The train step function



In [ ]:
# We will reuse this seed overtime (so it's easier)
num_examples_to_generate = 32
seed = tf.random.normal([num_examples_to_generate, noise_dim])


# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images, labels):
    
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape(persistent=True) as gen_tape, tf.GradientTape(persistent=True) as disc_tape:
        
        generated_images = generator([noise, labels], training=True)
    
        real_output, real_feature = discriminator([images, labels], training=True)
        fake_output, fake_feature = discriminator([generated_images, labels], training=True)

        #gen_loss_GAN = generator_loss_GAN(fake_output)
        gen_loss_GAN = gen_loss = generator_loss_FEATURE(real_feature, fake_feature)
        gen_loss_CYCLIC = generator_loss_CYCLIC(generated_images, images)        
        

        gen_loss = gen_loss_GAN+0.1*gen_loss_CYCLIC
        
        disc_loss = discriminator_loss(real_output, fake_output)

    
    for i in range(1):
        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    d_train_loss(disc_loss)
    g_train_loss(gen_loss)
    d_train_accuracy_on_real(tf.ones_like(real_output), real_output)
    d_train_accuracy_on_fake(tf.zeros_like(fake_output), fake_output)
    
    
    
    return gen_loss, disc_loss

In [ ]:
"""
checkpoint_dir = './condGAN/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
"""

def train(dataset, epochs):
    it = iter(test_dataset)
    X_test, label_test = next(it)
    
    for epoch in range(epochs):
        
        start = datetime.datetime.now()
            
        for image_batch, label_batch in train_dataset:
            
            g_loss, d_loss = train_step(image_batch, label_batch)
            
        with train_summary_writer.as_default():
            tf.summary.scalar('d_loss', d_train_loss.result(), step=epoch)
            tf.summary.scalar('g_loss', g_train_loss.result(), step=epoch)            
            tf.summary.scalar('d_accuracy_on_fake', d_train_accuracy_on_fake.result(), step=epoch)
            tf.summary.scalar('d_accuracy_on_real', d_train_accuracy_on_real.result(), step=epoch)            
                  
            
        # Produce images for the GIF as we go
        display.clear_output(wait=True)
        generate_and_save_images_conditional(generator,
                                 epoch + 1,
                                 seed, X_test, label_test, maxval)
        
        
        """
        
        # Save a checkpoint of the model every 10 epochs
        if (epoch + 1) % 10 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
        
        """

            

        print ('Time for epoch {0} is {1} sec'.format(epoch + 1, datetime.datetime.now()-start))

    # Generate after the final epoch
    #display.clear_output(wait=True)
    
    #generate_and_save_images_conditional(generator,
    #                           epochs,
    #                           seed, X_test, label_test, maxval)

In [ ]:
%tensorboard --logdir ./condGAN/logs/gradient_tape

In [ ]:
EPOCHS = 20
train(train_dataset, EPOCHS)